<a href="https://colab.research.google.com/github/edsolibet/tinymL/blob/main/sine_ml_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
!pip install tensorflow==2.4.1

In [ ]:
import tensorflow as tf

from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m

In [ ]:
# generate data
samples = 1000
seed = 1337
np.random.seed(seed)
tf.random.set_seed(seed)

x_val = np.random.uniform(low = 0, high = 2*m.pi, size = samples)
np.random.shuffle(x_val)
y_val = np.sin(x_val)

plt.plot(x_val, y_val, 'b.')
plt.show()

In [ ]:
# add noise
y_val += 0.15 * np.random.randn(*y_val.shape)

plt.plot(x_val, y_val, 'b.')
plt.show()

In [ ]:
# split data
train_split = int(0.6 * samples)
test_split = int(0.2 * samples + train_split)

x_train, x_validate, x_test = np.split(x_val, [train_split, test_split])
y_train, y_validate, y_test = np.split(y_val, [train_split, test_split])

assert (x_train.size + x_validate.size + x_test.size) == samples

plt.plot(x_train, y_train, 'b.', label = 'train')
plt.plot(x_validate, y_validate, 'r.', label = 'validate')
plt.plot(x_test, y_test, 'y.', label = 'test')
plt.legend()
plt.show()

In [ ]:
# define model architecture
model_1 = tf.keras.Sequential()

model_1.add(keras.layers.Dense(16, activation = 'relu', input_shape = (1,)))
model_1.add(keras.layers.Dense(16, activation = 'relu'))
model_1.add(keras.layers.Dense(1))
model_1.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

model_1.summary()


In [ ]:
# training them model
history_1 = model_1.fit(x_train, y_train, epochs = 1000, batch_size = 64,
                        validation_data = (x_validate, y_validate))

In [ ]:
loss = history_1.history['loss']
val_loss = history_1.history['val_loss']
mae = history_1.history['mae']
val_mae = history_1.history['val_mae']
epochs = range(1, len(loss) + 1)
skip = 0

plt.figure(figsize = (12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs[skip:], loss[skip:], 'g.', label = 'training loss')
plt.plot(epochs[skip:], val_loss[skip:], 'b.', label = 'validation loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(epochs[skip:], mae[skip:], 'g.', label = 'training mae')
plt.plot(epochs[skip:], val_mae[skip:], 'b.', label = 'validation mae')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# prediction

predictions = model_1.predict(x_test)

plt.clf()
plt.plot(x_test, y_test, 'b.', label = 'actual')
plt.plot(x_test, predictions, 'r.', label = 'predicted')
plt.legend()
plt.show()

In [ ]:
# convert model
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_generator():
  for value in x_test:
    yield [np.array(value, dtype = np.float32, ndmin = 2)]
converter.representative_dataset = representative_dataset_generator
tflite_model = converter.convert()

#save model
open("sine_model.tflite", 'wb').write(tflite_model)

In [ ]:
# instantitate an interpreter
sine_model = tf.lite.Interpreter('sine_model.tflite')
#allocate memory
sine_model.allocate_tensors()
#get indexes of the input and output tensors
sine_model_input_index = sine_model.get_input_details()[0]['index']
sine_model_output_index = sine_model.get_output_details()[0]['index']

# create arrays to store the results
sine_model_predictions = []

# run each model's interpreter for each value and store the results in arrays
for x in x_test:
  # create 2D tensor wrapping current x value
  x_value = tf.convert_to_tensor([[x]], dtype = np.float32)
  # write the value to the input tensor
  sine_model.set_tensor(sine_model_input_index, x_value)
  #run inference
  sine_model.invoke()
  # read the prediction from output tensor
  sine_model_predictions.append(sine_model.get_tensor(sine_model_output_index)[0])

plt.clf()
plt.plot(x_test, y_test, 'bo', label = 'Actual')
plt.plot(x_test, predictions, 'ro', label = 'Original predictions')
plt.plot(x_test, sine_model_predictions, 'gx', label = 'lite predictions')
plt.legend()
plt.show()

In [ ]:
model_size = os.path.getsize('sine_model.tflite')
print("Quantized model is %d bytes" % model_size)

In [ ]:
!apt-get -qq install xxd
!xxd -i sine_model.tflite > sine_model.cc
!cat sine_model.cc